# Prepare CSIS LPI data for publication

In [1]:
import glob
import os
import pandas as pd

# 2011-2018 data
lpi_path1 = "/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/"
# 2019-present data
lpi_path2 = "/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/"
dest_path1 = "/media/greg/jrn-DataProducts/JORNADA_IM/WIP_packages/210413002_CSIS_lpi_gap/"
#dest_path2 = "/media/greg/jrn-DataProducts/JORNADA_IM/WIP_packages/210413005_CSIS_overhead_cover/"

## Load the earlier DAT and CSV files

In [2]:
fns = [glob.glob(os.path.join(lpi_path1,'CSIS_LPI*.{0}'.format(e)), recursive=False) for e in ['csv', 'dat', 'DAT']]
fns = sum(fns, []) # In case we end up with list of lists
print('number LPI files in folder: ' + str(len(fns)))
fns

number LPI files in folder: 15


['/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_12F.csv',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_12S.csv',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_13S.csv',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_11F.csv',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_13F.dat',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_14F.dat',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_14S.dat',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_15F.dat',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_15S.dat',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_16F.dat',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis

In [3]:
df_dat = pd.read_fwf(fns[10], skiprows=44)
df_csv = pd.read_csv(fns[1])
print(df_dat.columns)
print(df_csv.columns)

Index(['study', 'year', 's', 'date', 'bk', 'p', 't', 'pt', 'top', 'ts', 'L1',
       'L1s', 'L2', 'L2s', 'L3', 'L3s', 'SL', 'SLs', 'tH', 'tFrm', 'tP',
       '1H 1Frm', '1P', '2H 2Frm', '2P', '3H 3Frm', '3P', 'sH sFrm', 'sP',
       'rIn', 'readerName', 'Unnamed: 31', 'dIn', 'dataentryName',
       'Unnamed: 34', 'Unnamed: 35'],
      dtype='object')
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'topLayer', 'topLayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilSurface',
       'soilSurface_status', 'comment', 'Recorder', 'Date Entered',
       'Date Verified', 'Intials'],
      dtype='object')


In [4]:
colmap_1 = {'s':'season','bk':'block','p':'plot','t':'transect','pt':'point',
            'top':'toplayer','ts':'toplayer_status',
            'L1':'layer1','L1s':'layer1_status',
            'L2':'layer2','L2s':'layer2_status',
            'L3':'layer3','L3s':'layer3_status',
            'SL':'soilsurface','SLs':'soilsurface_status',
            'tH':'toplayer_habit','tFrm':'toplayer_form','tP':'toplayer_carbon',
            '1H':'layer1_habit','1Frm':'layer1_form','1P':'layer1_carbon',
            '2H':'layer2_habit','2Frm':'layer2_form','2P':'layer2_carbon',
            '3H':'layer3_habit','3Frm':'layer3_form','3P':'layer3_carbon',
            'sH':'soilsurface_habit','sFrm':'soilsurface_form','sP':'soilsurface_carbon'}

colmap_2 = {'Date':'date','Block':'block','Plot':'plot',
            'topLayer':'toplayer','topLayer_status':'toplayer_status',
            'soilSurface':'soilsurface','soilSurface_status':'soilsurface_status',
            'Comment':'comment'}

In [5]:
# Read in the text files
frames = []

for f in fns:
    print(f)
    # Get number of lines to skip, use a windoze encoding for these old files
    with open(f, 'r', encoding='cp1252') as fo:
        lines = [line.rstrip() for line in fo]
    start_match = pd.Series(lines).str.contains('^CSIS-LPI')
    skip_no = start_match[start_match].index.values.min()
    print(skip_no)
    # Read csv and append to frames list (note these csvs have extra spaces)
    if skip_no==45:
        # These are old dat files and the header doesn't really match the columns - 
        # some right, some left justified. Steps below parse and correct col headers
        col_raw = pd.read_fwf(f, skiprows=(skip_no-1), nrows=0,
                              index_col=False, encoding='cp1252').columns.tolist()
        cols = [c.replace('Unnamed: ', 'Unnamed:') for c in col_raw]
        # join/splits to split multiple column headers read in as one
        cols = [' '.join(c.split()) for c in cols] # drop multiple spaces
        cols = ' '.join(cols).split(' ')  # split on any single spaced col header
        # Now read past the header row and use the corrected header
        # Big infer rows param to account for some unpredictable errors in data entry
        df = pd.read_fwf(f, skiprows=(skip_no), infer_nrows=10000, na_values=['.'], names=cols,
                         index_col=False, encoding='cp1252')
        # Rename some columns
        df = df.rename(columns=colmap_1)
        print(df.shape)
        print(df.columns)
    else:
        # Read file and rename
        df = pd.read_csv(f, na_values=['.'], encoding='cp1252')
        df = df.rename(columns=colmap_2)
        print(df.shape)
        print(df.columns)

    frames.append(df)

# Concatenate dfs into one
df1 = pd.concat(frames, axis=0, ignore_index=True)
# Set dates
df1.date = pd.to_datetime(df1.date, format='%m/%d/%Y')

/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_12F.csv
1


(6120, 23)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'comment', 'Recorder', 'Date Entered',
       'Date Verified', 'Intials'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_12S.csv
1
(6120, 23)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'comment', 'Recorder', 'Date Entered',
       'Date Verified', 'Intials'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_13S.csv
1
(6120, 23)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       't

/tmp/ipykernel_257792/980981662.py:24: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_fwf(f, skiprows=(skip_no), infer_nrows=10000, na_values=['.'], names=cols,


(17884, 41)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'toplayer_habit', 'toplayer_form',
       'toplayer_carbon', 'layer1_habit', 'layer1_form', 'layer1_carbon',
       'layer2_habit', 'layer2_form', 'layer2_carbon', 'layer3_habit',
       'layer3_form', 'layer3_carbon', 'soilsurface_habit', 'soilsurface_form',
       'soilsurface_carbon', 'rIn', 'readerName', 'Unnamed:31', 'dIn',
       'dataentryName', 'Unnamed:34', 'Unnamed:35', 'Unnamed:36'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_15S.dat
45


/tmp/ipykernel_257792/980981662.py:24: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_fwf(f, skiprows=(skip_no), infer_nrows=10000, na_values=['.'], names=cols,


...

(17888, 40)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'toplayer_habit', 'toplayer_form',
       'toplayer_carbon', 'layer1_habit', 'layer1_form', 'layer1_carbon',
       'layer2_habit', 'layer2_form', 'layer2_carbon', 'layer3_habit',
       'layer3_form', 'layer3_carbon', 'soilsurface_habit', 'soilsurface_form',
       'soilsurface_carbon', 'rIn', 'readerName', 'Unnamed:31', 'dIn',
       'dataentryName', 'Unnamed:34', 'Unnamed:35'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_17S.dat
45
(17902, 41)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
      

/tmp/ipykernel_257792/980981662.py:24: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_fwf(f, skiprows=(skip_no), infer_nrows=10000, na_values=['.'], names=cols,


(17895, 40)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'toplayer_habit', 'toplayer_form',
       'toplayer_carbon', 'layer1_habit', 'layer1_form', 'layer1_carbon',
       'layer2_habit', 'layer2_form', 'layer2_carbon', 'layer3_habit',
       'layer3_form', 'layer3_carbon', 'soilsurface_habit', 'soilsurface_form',
       'soilsurface_carbon', 'rIn', 'readerName', 'Unnamed:32', 'dIn',
       'dataentryName', 'Unnamed:35', 'Unnamed:36'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_data/CSIS_LPI_18F.DAT
45


/tmp/ipykernel_257792/980981662.py:24: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_fwf(f, skiprows=(skip_no), infer_nrows=10000, na_values=['.'], names=cols,


(17896, 40)
Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'toplayer_habit', 'toplayer_form',
       'toplayer_carbon', 'layer1_habit', 'layer1_form', 'layer1_carbon',
       'layer2_habit', 'layer2_form', 'layer2_carbon', 'layer3_habit',
       'layer3_form', 'layer3_carbon', 'soilsurface_habit', 'soilsurface_form',
       'soilsurface_carbon', 'rIn', 'readerName', 'Unnamed:32', 'dIn',
       'dataentryName', 'Unnamed:35', 'Unnamed:36'],
      dtype='object')


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205525 entries, 0 to 205524
Data columns (total 52 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   study               205525 non-null  object        
 1   year                205525 non-null  int64         
 2   season              205525 non-null  object        
 3   date                205525 non-null  datetime64[ns]
 4   block               205525 non-null  int64         
 5   plot                205525 non-null  object        
 6   transect            205525 non-null  int64         
 7   point               205525 non-null  int64         
 8   toplayer            204984 non-null  object        
 9   toplayer_status     77561 non-null   object        
 10  layer1              81364 non-null   object        
 11  layer1_status       7820 non-null    object        
 12  layer2              5432 non-null    object        
 13  layer2_status       229 non-n

In [7]:
df1.head()

,study,year,season,date,block,plot,transect,point,toplayer,toplayer_status,...,dIn,dataentryName,Unnamed:36,Unnamed:37,Unnamed:38,Unnamed:31,Unnamed:34,Unnamed:35,Unnamed:29,Unnamed:30
0,CSIS-LPI,2012,F,2012-09-06,1,A,1,0,NONE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CSIS-LPI,2012,F,2012-09-06,1,A,1,50,NONE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CSIS-LPI,2012,F,2012-09-06,1,A,1,100,NONE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CSIS-LPI,2012,F,2012-09-06,1,A,1,150,NONE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CSIS-LPI,2012,F,2012-09-06,1,A,1,200,NONE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Read in recent Excel files

In [8]:
# first list the files
fns = glob.glob(os.path.join(lpi_path2,'Data_CSIS_LPI*.xlsx'), recursive=False)
#fns = sum(fns, []) # In case we end up with list of lists
print('number LPI files in folder: ' + str(len(fns)))
# Drop the 2023 data for now
fns = [f for f in fns if '2023' not in f]
fns

number LPI files in folder: 9


['/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2019S.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2019F.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2020F.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2020S.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2021F.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2021S.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2022S.xlsx',
 '/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2022F.xlsx']

In [9]:
colmap_3 = {'intercept':'point','top_layer':'toplayer','layer-1':'layer1',
            'layer-2':'layer2','layer-3':'layer3','soil_surface':'soilsurface'}

In [10]:
# Read in the excel files
frames = []

for f in fns:
    print(f)
    # Find the sheet we need (they vary year to year)
    sheet = [s for s in pd.ExcelFile(f).sheet_names if 'pq' in s and 'LPI' in s][0]
    print(sheet)
    # Header rows are variable. Read first 20 lines and find header row (contains 
    # 'year') to set skiprows parameter
    df_head = pd.read_excel(f, sheet, nrows=20)
    header_loc = df_head[df_head == 'year'].dropna(axis=1, how='all').dropna(how='all')
    if header_loc.empty:
        skip = None # Don't skip anything if we get an empty df
    else:
        skip = header_loc.index.item() + 1
    # Now read the file
    df = pd.read_excel(f, sheet_name=sheet, skiprows=skip)
    # Rename columns and replace values
    df = df.rename(columns=colmap_3)
    df.season = df.season.replace({'Spring':'S','Fall':'F'})
    df['study'] = 'CSIS-LPI'
    print(df.shape)
    print(df.columns)

    frames.append(df)

# Concatenate dfs into one
df2 = pd.concat(frames, axis=0, ignore_index=True)
    

/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2019S.xlsx


pq_2019S_LPI
(17886, 14)
Index(['year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'layer1', 'layer2', 'layer3', 'soilsurface', 'comment',
       'study'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2019F.xlsx
pq_2019F_LPI
(17892, 14)
Index(['year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'layer1', 'layer2', 'layer3', 'soilsurface', 'comment',
       'study'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2020F.xlsx
pq_2020F_LPI
(17460, 14)
Index(['year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'layer1', 'layer2', 'layer3', 'soilsurface', 'comment',
       'study'],
      dtype='object')
/media/greg/jrn-DataArchive/Data_ENT/LTER/_Entry/CSIS/csis-VegTran/_compiled/Data_CSIS_LPI-Gap_2020S.xlsx
pq_2020S_LPI
(17907, 14)
Index(['year', 'season', 'd

In [11]:
df2.head()

,year,season,date,block,plot,transect,point,toplayer,layer1,layer2,layer3,soilsurface,comment,study
0,2019,S,2019-04-12,1,A,1,0.0,NONE,NaN,NaN,NaN,S,NaN,CSIS-LPI
1,2019,S,2019-04-12,1,A,1,25.0,NONE,NaN,NaN,NaN,S,NaN,CSIS-LPI
2,2019,S,2019-04-12,1,A,1,50.0,NONE,NaN,NaN,NaN,S,NaN,CSIS-LPI
3,2019,S,2019-04-12,1,A,1,75.0,NONE,NaN,NaN,NaN,S,NaN,CSIS-LPI
4,2019,S,2019-04-12,1,A,1,100.0,NONE,NaN,NaN,NaN,S,NaN,CSIS-LPI


## Merge the 2 dataframes

In [12]:
df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.columns

Index(['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'comment', 'Recorder', 'Date Entered',
       'Date Verified', 'Intials', 'toplayer_habit', 'toplayer_form',
       'toplayer_carbon', 'layer1_habit', 'layer1_form', 'layer1_carbon',
       'layer2_habit', 'layer2_form', 'layer2_carbon', 'layer3_habit',
       'layer3_form', 'layer3_carbon', 'soilsurface_habit', 'soilsurface_form',
       'soilsurface_carbon', 'rIn', 'readerName', 'Unnamed:32', 'Unnamed:33',
       'dIn', 'dataentryName', 'Unnamed:36', 'Unnamed:37', 'Unnamed:38',
       'Unnamed:31', 'Unnamed:34', 'Unnamed:35', 'Unnamed:29', 'Unnamed:30'],
      dtype='object')

In [13]:
keep = ['study', 'year', 'season', 'date', 'block', 'plot', 'transect', 'point',
       'toplayer', 'toplayer_status', 'layer1', 'layer1_status', 'layer2',
       'layer2_status', 'layer3', 'layer3_status', 'soilsurface',
       'soilsurface_status', 'comment']
df = df.loc[:,keep]

### Make a few corrections before writing

In [14]:
{c:df[c].unique() for c in ['toplayer', 'layer1', 'layer2', 'layer3']}

{'toplayer': array(['NONE', 'PRGL', 'SPOR1', 'XASA', 'ZIGR', 'BOER', 'BOER ', 'MUPO',
        'ARIS1', 'CABA', 'XASA ', 'YUEL', 'PRGL ', 'MUPO ', 'SPOR1 ',
        'SAKA', 'KRLA', 'CHIN', 'CRPO', 'DAPU', 'ERAB', 'CAJA', 'MISS',
        'CRPO ', 'SOEL', nan, 'EPTR', 'EUAL', 'LATR', 'SPFL', 'NONE ',
        'ATCA', 'XANT1', 'SPCO', 'BOBA', 'MOCE', 'EUMI', 'BOSP', 'TILA',
        'PORT1', 'KAPA', 'ALIN', 'BOAR', 'PAHI', 'PEPA', 'ARAD', 'PEAN',
        'SPAI', 'BRFA', 'TAAU', 'TRTE', 'EUSE', 'DECO', 'VEEN', 'EUSR',
        'BOUT1', 'EUDE', 'BRAR', 'SELE', 'EUGL', 'PECT1', 'KALL1', 'POOL',
        'MUSQ', 'HOGL', 'TALI1', 'EUPH1', 'TECO', 'ARPU', 'DIWI', 'SPIN',
        'BAMU', 'ARHA', 'ARPE', 'DANA', 'ARNE', 'SAHA', 'PROB1', 'ZIAC',
        'POMU', 'HODR', 'CONMD', 'BAAB', 'ERPE', 'SPCR', 'APRA', 'UKGRS',
        'ARWR', 'PRGLS', 'AMPA', 'BOER1', 'BRAC1', 'ERLE', 'ERTE', 'ASAL',
        'NAHI', 'CAHU', 'TAAN', 'ARPA', 'DYPE', 'CRYP1', 'ASTE', 'DYAC',
        'SILE', 'HEHU', 'NYCT1', 'DEPI'

In [15]:
# Unique plant status patterns - change D to S or B to see others
print(df.loc[df.toplayer.str.contains('[A-Z,1-9]{4,}D$', na=False), 'toplayer'].unique())
print(df.loc[df.layer1.str.contains('[A-Z,1-9]{4,}D$', na=False), 'layer1'].unique())
print(df.loc[df.layer2.str.contains('[A-Z,1-9]{4,}D$', na=False), 'layer2'].unique())
print(df.loc[df.layer3.str.contains('[A-Z,1-9]{4,}D$', na=False), 'layer3'].unique())
print(df.loc[df.soilsurface.str.contains('[A-Z,1-9]{4,}D$', na=False), 'soilsurface'].unique())

['CONMD' 'PRGLD' 'XASAD' 'SPFLD' 'SPOR1D' 'BOERD' 'ARIS1D' 'SELED' 'MUPOD'
 'CRPOD' 'DAPUD' 'SAKAD' 'DEPID' 'YUELD' 'ARPUD' 'NAHID' 'XANT1D' 'LIVED'
 'DIWID' 'APRAD' 'SPCOD' 'ZIGRD' 'CRYP1D' 'CRCRD' 'BAMUD' 'CAJAD' 'KRLAD'
 'MEPUD' 'BOBAD' 'TILAD' 'SOELD' 'STEXD' 'BOIND' 'MUSQD' 'PAHID' 'ARPED'
 'EPTRD' 'BOSPD' 'ARADD' 'CABAD' 'BOER1D' 'PECT1D' 'BRAC1D' 'ACWRD'
 'SPCRD' 'MOCED' 'VEEND' 'AMPAD' 'AMFID' 'KAPAD' 'ZIACD']
['CONMD' 'PRGLD' 'MUPOD' 'BOERD' 'SPOR1D' 'CRPOD' 'SELED' 'DAPUD' 'SPFLD'
 'DEPID' 'XANT1D' 'APRAD' 'DIWID' 'XASAD' 'SPCOD' 'ARIS1D' 'BOBAD' 'TILAD'
 'BOSPD' 'MUSQD' 'PAHID' 'SOELD' 'BRAC1D' 'ARPED' 'ARADD' 'SAKAD' 'VEEND'
 'PECT1D' 'AMPAD']
['BOERD' 'PRGLD' 'MUPOD' 'XANT1D' 'XASAD' 'DEPID' 'SPFLD' 'BOBAD' 'BRAC1D'
 'TILAD' 'PAHID' 'BOSPD']
['BOERD' 'TILAD']
['CONMD' 'SPOR1D' 'SPFLD' 'YUELD' 'BOERD' 'ARIS1D' 'DAPUD' 'XANT1D'
 'XASAD' 'BOBAD' 'SELED' 'SPCOD' 'ARPUD' 'MUPOD']


In [16]:
# There are three cover statuses appended to some codes D, S, B
# D=dead, S=seedling, B=black (last years growth?)
# Find these and split out to the appropriate status column
# Also remove some wacky spaces and lowercase letters
for c in ['toplayer', 'layer1', 'layer2', 'layer3', 'soilsurface']:
    # Find column values longer than 4, ending in D, S, B & modify
    # column_status, then shorten the column value itself
    testD = df[c].str.contains('[A-Z,1-9]{4,}D$', na=False)
    df.loc[testD, c + '_status'] = 'D'
    df.loc[testD, c] = df.loc[testD, c].str[0:-1]
    testB = df[c].str.contains('[A-Z,1-9]{4,}B$', na=False)
    df.loc[testB, c + '_status'] = 'B'
    df.loc[testB, c] = df.loc[testB, c].str[0:-1]
    testS = df[c].str.contains('[A-Z,1-9]{4,}S$', na=False)
    df.loc[testS, c + '_status'] = 'S'
    df.loc[testS, c] = df.loc[testS, c].str[0:-1]
    # Wacky spaces and lowercase
    df[c] = df[c].str.strip()
    df[c] = df[c].str.upper()



In [17]:
# Now write
df.to_csv(os.path.join(dest_path1,'jrn413002_lpi_data.csv'), index=False, na_rep='NA')


In [18]:
df.point.unique()
df.date.max()

Timestamp('2022-08-22 00:00:00')

In [19]:
df.groupby(['block', 'plot', 'transect']).size().reset_index(name='Freq')

,block,plot,transect,Freq
0,1,A,1,1903
1,1,A,2,1862
2,1,A,3,1696
3,1,B,1,1903
4,1,B,2,1957
...,...,...,...,...
175,15,C,2,1860
176,15,C,3,1692
177,15,D,1,2003
178,15,D,2,1957


In [20]:
df.isna().sum()


study                      0
year                       0
season                     0
date                       0
block                      0
plot                       0
transect                   0
point                    121
toplayer                 670
toplayer_status       245420
layer1                195868
layer1_status         328271
layer2                327016
layer2_status         337466
layer3                337180
layer3_status         337744
soilsurface              938
soilsurface_status    334956
comment               337001
dtype: int64